In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
path = "..\\Output\\Clean Cities.csv"

df = pd.read_csv(path)

df

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
0,-54.800000,-68.30000,ushuaia,ar,42.80,52.0,75.0,16.11,1.599706e+09
1,-33.650000,115.33000,busselton,au,57.00,67.0,0.0,5.99,1.599707e+09
2,-25.170000,45.05000,beloha,mg,70.97,64.0,48.0,10.02,1.599707e+09
3,28.320000,109.72000,jishou,cn,80.22,79.0,79.0,1.92,1.599707e+09
4,8.570000,81.23000,trincomalee,lk,83.71,71.0,91.0,18.12,1.599707e+09
...,...,...,...,...,...,...,...,...,...
278,40.430000,71.72000,komsomolskiy,ru,69.80,35.0,0.0,2.24,1.599707e+09
279,1.538881,-170.53697,saleaula,ws,NaN,NaN,NaN,NaN,NaN
280,61.990000,-49.67000,paamiut,gl,39.99,78.0,88.0,22.97,1.599707e+09
281,-0.910000,122.88000,biak,id,82.15,73.0,28.0,8.23,1.599707e+09


In [3]:
locations = df[["lat", "long"]]

weight = df["humidity"]

In [4]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights = weight,
    max_intensity=20, point_radius=5
)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
filtered_df = df.loc[(df["wind speed"]<=10) & (df["max temp"]>=70) & (df["max temp"]<=80) & (df["cloudiness"]==0) & 
                    (df["humidity"]<=80)].dropna()

filtered_df

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
53,-12.74,-60.15,vilhena,br,78.42,28.0,0.0,3.09,1.599707e+09
157,29.58,-9.50,tiznit,ma,73.11,41.0,0.0,2.86,1.599707e+09
183,41.30,13.03,sabaudia,it,73.99,70.0,0.0,3.00,1.599707e+09
205,-11.38,43.28,mitsamiouli,km,76.50,75.0,0.0,5.37,1.599707e+09
220,30.48,-115.95,san quintin,mx,73.35,66.0,0.0,8.52,1.599707e+09
260,39.21,54.59,gumdag,tm,72.64,21.0,0.0,3.98,1.599707e+09
271,41.12,80.26,aksu,cn,72.54,38.0,0.0,4.79,1.599707e+09


In [6]:
filt_locations = filtered_df[["lat","long"]]

filt_weight = filtered_df["humidity"]

In [7]:
fig2 = gmaps.figure(center=(35,40), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(
    filt_locations, weights=filt_weight,
    max_intensity=60, point_radius=20
)
fig2.add_layer(heatmap_layer)
plt.savefig("..\Output\FilteredHeatMap.png")
fig2

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"types":"lodging",
         "radius":5000,
         "key":g_key}

for index, row in filtered_df.iterrows():
    lat = row["lat"]
    lng = row["long"]
    cityname= row["city"]
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url,params=params).json()
    
    try:
        print(f"Looking for Hotels in {index}: {cityname}")
        filtered_df.loc[index, "hotel name"] = response["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("--------Done!")

Looking for Hotels in 53: vilhena
Looking for Hotels in 157: tiznit
Hotel not found... skipping.
Looking for Hotels in 183: sabaudia
Looking for Hotels in 205: mitsamiouli
Looking for Hotels in 220: san quintin
Looking for Hotels in 260: gumdag
Hotel not found... skipping.
Looking for Hotels in 271: aksu
--------Done!


In [12]:
filtered_df

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date,hotel name
53,-12.74,-60.15,vilhena,br,78.42,28.0,0.0,3.09,1.599707e+09,Hotel Mirage
157,29.58,-9.50,tiznit,ma,73.11,41.0,0.0,2.86,1.599707e+09,NaN
183,41.30,13.03,sabaudia,it,73.99,70.0,0.0,3.00,1.599707e+09,Oasi di Kufra
205,-11.38,43.28,mitsamiouli,km,76.50,75.0,0.0,5.37,1.599707e+09,Foyer ADM
220,30.48,-115.95,san quintin,mx,73.35,66.0,0.0,8.52,1.599707e+09,Old Mill Hotel
260,39.21,54.59,gumdag,tm,72.64,21.0,0.0,3.98,1.599707e+09,NaN
271,41.12,80.26,aksu,cn,72.54,38.0,0.0,4.79,1.599707e+09,Pudong Holiday Hotel


In [20]:
hotels_df = filtered_df[["lat","long","city", "country", "hotel name"]].dropna()

hotels_df

,lat,long,city,country,hotel name
53,-12.74,-60.15,vilhena,br,Hotel Mirage
183,41.30,13.03,sabaudia,it,Oasi di Kufra
205,-11.38,43.28,mitsamiouli,km,Foyer ADM
220,30.48,-115.95,san quintin,mx,Old Mill Hotel
271,41.12,80.26,aksu,cn,Pudong Holiday Hotel


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["lat", "long"]]

In [24]:

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig2.add_layer(markers)

fig2

Figure(layout=FigureLayout(height='420px'))